
# **Analytics 2 :** <font color=#DF4807>**Probabilisitc Models and Auto Complete**</font>

**Objectives**:

1. Pre-process the data
2. Add Start and end Tokens
3. Creating a vocabulary list of unique words
4. Calculate ngram frequencies
5. Calcualte Conditional Probabilities with k-smoothing




In [4]:
#import libraries
import nltk
from nltk.corpus import twitter_samples
nltk.download('twitter_samples|')
import re

[nltk_data] Error loading twitter_samples|: <urlopen error [Errno
[nltk_data]     11001] getaddrinfo failed>


## **Loading the Data:**

In [5]:
tweets = twitter_samples.strings('positive_tweets.json')
tweets[:5] #let's look at a sample

['#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)',
 '@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!',
 '@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!',
 '@97sides CONGRATS :)',
 'yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days']

## **Pre-processing text data:**

1. split data into sentences
2. split each sentence into tokens
3. additional pre-processing that you think would be benefitial.

In [6]:
#your code here
import spacy

nlp = spacy.load('en_core_web_sm')

all_sentences  = []
for each_sentence in tweets:
    sentence = []
    for each_word in each_sentence.split():
        # Converting to lower case if it's not starting with # or @
        if not each_word.startswith('#') or each_word.startswith('@'):
            sentence.append(each_word.lower())
        else:
            sentence.append(each_word)
        # sentence.append(each_word.text)
    # Inserting <start> and <end> in the beginning and the ending of the sentence.
    sentence.insert(0,'<start>')
    sentence.insert(len(sentence)+1,'<end>')
    # Adding all the sentences after tokenization 
    all_sentences.append(sentence)

## **Create Vocabulary list:**
We won't be using all the vocabulary in our dataset. Instead we will set a threshold and only use the vocabulary that appear more than that given threshold. This is to remove words which aren't important.

**Task 1:** First we need to calcualte word frequencies. Let's create a dictionary with all the unique words in the data and the frequency in which they appear.

In [7]:
word_freqs = {} #getting the frequency of all the words that occur in our data
#your code here:

#calculating the word frequency
for sentence in all_sentences:
    for each_word in sentence:
        # Checking if the words are already created.
        if each_word in  word_freqs.keys():
            word_freqs[each_word] += 1
        else:
            word_freqs[each_word] = 1

    

**Task 2:**  Now, we create our vocabulary using words above a certain frequency threshold.

In [37]:
# Empty list for closed vocabulary

#your code here
vocabulary = []
# Setting the upper Threshold and lower threshold
upper_threshold = 3500 #You might need to do some testing on this first to get an idea of how your data is distributed.


lower_threshold = 2

# Fetching all the vocab after 
vocabulary = [ key for key,value in word_freqs.items() if value < upper_threshold and value > lower_threshold]
print(vocabulary)




['#FollowFriday', 'for', 'being', 'top', 'engaged', 'members', 'in', 'my', 'community', 'this', 'week', ':)', 'hey', 'james!', 'how', ':/', 'please', 'call', 'our', 'contact', 'on', 'and', 'we', 'will', 'be', 'able', 'to', 'you', 'many', 'thanks!', 'had', 'a', 'listen', 'last', 'night', 'as', 'is', 'an', 'amazing', 'when', 'are', 'congrats', 'has', 'got', 'blue', 'fb', '15', 'days', '@bhaktisbanter', '@pallaviruhail', 'one', '#FlipkartFashionFriday', "don't", 'like', 'keep', 'lovely', 'waiting', 'hope', 'enjoy!', 'happy', 'friday!', '-', 'second', 'just', 'not', 'enough', 'time', 'but', 'new', 'must', 'jgh', ',', 'have', 'go', ':d', 'bye', 'act', 'of', 'am', 'calling', 'the', 'app', 'name', 'influencers', 'who', "wouldn't", 'love', 'these', 'big....juicy....selfies', 'follow', '@jnlazts', '&amp;', 'http://t.co/rcvcyyo0iq', 'u', 'back', 'so', 'already', 'know', "what's", 'great', 'opportunity', '12', '13', 'at', 'get', 'your', 'out', 'greetings', 'card', 'today', 'job', ':-)', "friend's

## **Dealing with Unknown Words:**

What happens when your model encounters a word which it has never seen before? In such a case, your model won't be able to make a prediction because there are no counts of that word. So, to be able to make predictions, we are going to include an "unknown" token.

In [9]:
#go through your sentences again and replace words with ("<unk>") if they  are not in our vocabulary list.
new_sentence = 'I love being Curious.'
print([each if each not in vocabulary else '<unk>' for each in new_sentence.split() ])



['I', '<unk>', '<unk>', 'Curious.']


## **Add Start and End Tokens:**

To be albe to calculate conditional probabilites for the first and last words in our sentences, we will inlclude 2 new tokens into our data. These are:

<'start'> and <'end'>.

For example:

I went to the store. --> [<'start'>, i , went, to, the, store, <'end'>]




## **bigram Frequencies:**

Create a dictionary for bigrams with a key (bigram) value (count) pair.

In [10]:
#task 1: add start and end tokens to the sentences
# all_sentences[0]
#task 2: Create a dictionary for with the bigrams frequencies i.e. a key (bigram) value (count) pair.

def create_n_gram(n):
    
    global all_sentences
    word_freqs = {}
    for sentence in all_sentences:
        for index,each_word in enumerate(sentence):
            ngram_words = " ".join(sentence[index:index+n])
            if ngram_words in  word_freqs.keys():
                
                word_freqs[ngram_words] += 1
            else:
                word_freqs[ngram_words] = 1
                
    return word_freqs

In [11]:
all_vocab = create_n_gram(2)

## **Ngram Probabilities with K-smoothing:**

The next step involves us converting the frequencies to probabilities. For this purpose we will use Function 1:
\
\
$$ \hat{P}(w_t | w_{t-n} \dots w_{t-1}) = \frac{Count(w_{t-n}\dots w_{t-1}, w_t)}{Count(w_{t-n}\dots w_{t-1})} \tag{1} $$
\
\
In other words, the probability of current word following the previous word(s) = count(prev word(s), current word)/ count(previous word(s). However, this can be probalematic when probabilites are zero. To avoid this, we use smooothing techniques, one of which is k-smoothing.
\
\
$$ \hat{P}(w_t | w_{t-n} \dots w_{t-1}) = \frac{Count(w_{t-n}\dots w_{t-1}, w_t) + k}{Count(w_{t-n}\dots w_{t-1}) + k|V|} \tag{2} $$
\
\
In this approach, we add a constant to the numerator and we add the same constant* length of vocabualry to the denominator.

In [31]:
#count of the times ":)" followed "thanks"
all_vocab['thanks :)']

all_thanks = [key for key in all_vocab.keys() if 'thanks' in key.split()[0]]
all_smile = [key for key in all_vocab.keys() if ':)' in key]

uni_gram_length = len(create_n_gram(1).keys())
bi_gram_length = len(all_vocab.keys())

print(f"The number times thanks occurs is: {all_vocab['thanks :)'] }")

#let's calculate the probability of getting a happy face ":)" after "thanks" with k smoothing
k = 0.01
prob_thanks_given_happy_face = all_vocab['thanks :)'] / len(all_thanks)
prob_thanks_given_happy_face
print(f"Without K-smoothing {prob_thanks_given_happy_face}")

prob_thanks_given_happy_face = all_vocab['thanks :)'] + k / (len(all_thanks) + (k*uni_gram_length))
prob_thanks_given_happy_face

print(f"With K-smoothing {prob_thanks_given_happy_face}")

The number times thanks occurs is: 55
Without K-smoothing 0.41353383458646614
With K-smoothing 55.0000339673913


In [40]:
create_n_gram(3)

{'<start> #FollowFriday @france_inte': 1,
 '#FollowFriday @france_inte @pkuchly57': 1,
 '@france_inte @pkuchly57 @milipol_paris': 1,
 '@pkuchly57 @milipol_paris for': 1,
 '@milipol_paris for being': 1,
 'for being top': 26,
 'being top engaged': 6,
 'top engaged members': 6,
 'engaged members in': 6,
 'members in my': 6,
 'in my community': 24,
 'my community this': 24,
 'community this week': 24,
 'this week :)': 25,
 'week :) <end>': 23,
 ':) <end>': 1843,
 '<end>': 5000,
 '<start> @lamb2ja hey': 1,
 '@lamb2ja hey james!': 1,
 'hey james! how': 1,
 'james! how odd': 1,
 'how odd :/': 1,
 'odd :/ please': 1,
 ':/ please call': 1,
 'please call our': 1,
 'call our contact': 1,
 'our contact centre': 1,
 'contact centre on': 1,
 'centre on 02392441234': 1,
 'on 02392441234 and': 1,
 '02392441234 and we': 1,
 'and we will': 3,
 'we will be': 5,
 'will be able': 3,
 'be able to': 6,
 'able to assist': 1,
 'to assist you': 1,
 'assist you :)': 1,
 'you :) many': 1,
 ':) many thanks!': 1,
 

In [46]:
def find_the_next_word(sentence):

    k = 0.01
    
    # Creating n gram based on the all arguments length
    all_vocab = create_n_gram(len(sentence.split()))

    # Placing all the arguments in a list
    all_words = sentence.split()

    # Getting all the words except last word
    early_words = " ".join(all_words[:-1])

    

    last_word = all_words[-1]
    
    # 
    all_text1 = [key for key in all_vocab.keys() if early_words in key]
    all_text2 = [key for key in all_vocab.keys() if last_word in key]


    k_smoothing_for_given_word = all_vocab[f"{early_words} {last_word}"] + k / (len(all_text1) + (k*uni_gram_length))
    print(f"K-smoothing {k_smoothing_for_given_word}")
    
    all_prob = {}
    for each_text in all_text1:
        if each_text.split()[0] == early_words:
            second_word = each_text.split()[1]
            prob_of_given_words = all_vocab[f"{early_words} {second_word}"] + k / (len(all_text1) + (k*uni_gram_length))
            all_prob[f"{early_words} {second_word}"] = prob_of_given_words
    top_recommend = (sorted(all_prob.items(),key=lambda x: x[1],reverse=True))[:5]
    for each_recommendation in top_recommend:
        print(each_recommendation)
find_the_next_word('please call')
# find_the_next_word('this','week',':)')


K-smoothing 2.000033738191633
('please :)', 10.000033738191632)
('please follow', 8.000033738191632)
('please do', 6.000033738191633)
('please dm', 4.000033738191633)
('please employers!', 3.000033738191633)
